<a href="https://colab.research.google.com/github/firekind/END3/blob/main/session-4/imdb-lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm', include_lengths=True)
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
print(vars(train_data.examples[0]))

{'text': ['While', 'this', 'was', 'a', 'better', 'movie', 'than', '101', 'Dalmations', '(', 'live', 'action', ',', 'not', 'animated', 'version', ')', ',', 'I', 'think', 'it', 'still', 'fell', 'a', 'little', 'short', 'of', 'what', 'Disney', 'could', 'do', '.', 'It', 'was', 'well', '-', 'filmed', ',', 'the', 'music', 'was', 'more', 'suited', 'to', 'the', 'action', ',', 'and', 'the', 'effects', 'were', 'better', 'done', '(', 'compared', 'to', '101', ')', '.', 'The', 'acting', 'was', 'perhaps', 'better', ',', 'but', 'then', 'the', 'human', 'characters', 'were', 'given', 'far', 'more', 'appropriate', 'roles', 'in', 'this', 'sequel', ',', 'and', 'Glenn', 'Close', 'is', 'really', 'not', 'to', 'be', 'missed', ',', 'as', 'in', 'the', 'first', 'movie', '.', 'She', 'makes', 'it', 'shine', '.', 'Her', 'poor', 'lackey', 'and', 'the', 'overzealous', 'furrier', 'sidekicks', 'are', 'wonderful', 'characters', 'to', 'play', 'off', 'of', ',', 'and', 'they', 'add', 'to', 'the', 'spectacle', 'Disney', 'has

In [5]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [7]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [8]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [9]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203883), (',', 193363), ('.', 166762), ('and', 110063), ('a', 109530), ('of', 100924), ('to', 93842), ('is', 76734), ('in', 61619), ('I', 54378), ('it', 53503), ('that', 49231), ('"', 44186), ("'s", 43472), ('this', 42441), ('-', 37108), ('/><br', 35778), ('was', 35097), ('as', 30575), ('with', 30303)]


In [10]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [11]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [12]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch=True,
    device = device)

In [13]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        embedded = self.embedding(text)        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu())
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        
        return self.fc(hidden.squeeze(0)).view(-1)

In [14]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,867,049 trainable parameters


In [16]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [17]:
criterion = nn.BCEWithLogitsLoss()

In [18]:
model = model.to(device)
criterion = criterion.to(device)

In [19]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        text, text_lengths = batch.text
                
        predictions = model(text, text_lengths)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [23]:
N_EPOCHS = 30

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 18s
	Train Loss: 0.690 | Train Acc: 52.53%
	 Val. Loss: 0.685 |  Val. Acc: 55.93%
Epoch: 02 | Epoch Time: 0m 18s
	Train Loss: 0.667 | Train Acc: 59.47%
	 Val. Loss: 0.620 |  Val. Acc: 65.35%
Epoch: 03 | Epoch Time: 0m 18s
	Train Loss: 0.589 | Train Acc: 69.14%
	 Val. Loss: 0.580 |  Val. Acc: 70.40%
Epoch: 04 | Epoch Time: 0m 17s
	Train Loss: 0.548 | Train Acc: 72.94%
	 Val. Loss: 0.538 |  Val. Acc: 73.51%
Epoch: 05 | Epoch Time: 0m 17s
	Train Loss: 0.503 | Train Acc: 75.96%
	 Val. Loss: 0.500 |  Val. Acc: 75.19%
Epoch: 06 | Epoch Time: 0m 18s
	Train Loss: 0.481 | Train Acc: 77.63%
	 Val. Loss: 0.501 |  Val. Acc: 77.49%
Epoch: 07 | Epoch Time: 0m 18s
	Train Loss: 0.455 | Train Acc: 79.37%
	 Val. Loss: 0.464 |  Val. Acc: 78.26%
Epoch: 08 | Epoch Time: 0m 18s
	Train Loss: 0.415 | Train Acc: 81.74%
	 Val. Loss: 0.436 |  Val. Acc: 79.51%
Epoch: 09 | Epoch Time: 0m 17s
	Train Loss: 0.402 | Train Acc: 82.62%
	 Val. Loss: 0.434 |  Val. Acc: 79.92%
Epoch: 10 | Epoch T

In [24]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.369 | Test Acc: 85.17%
